In [15]:
from torchtext import data
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
from dataset import DataHandler, BertField
import torch.nn as nn
import torch
from model import BiAttention, EncoderRNN, SelfAttention
import os

In [2]:
train_examples_path = './train_examples.pt'
val_examples_path = './val_examples.pt'

In [3]:
tokenizer = BertTokenizer.from_pretrained('./bert-base-uncased-vocab.txt')

bert_field = BertField(tokenizer)
multi_bert_field = data.NestedField(bert_field)

word_field = data.Field(batch_first=True, sequential=True, tokenize=tokenizer.tokenize, lower=True) # query
multi_word_field = data.NestedField(word_field) 

raw = data.RawField()
raw.is_target = False

label_field = data.Field(sequential=False, is_target=True, use_vocab=False)

dict_field = {
    'id': ('id', raw),
    'supports': [('s_glove', multi_word_field), ('s_bert', multi_bert_field)],
    'query': [('q_glove', word_field), ('q_bert', bert_field)],
    'answer': [('a_glove', word_field), ('a_bert', bert_field)],
    'candidates': [('c_glove', multi_word_field), ('c_bert', multi_bert_field)],
    'label': ('label', label_field)

}

In [4]:
data_handler = DataHandler(train_examples_path, val_examples_path, dict_field, word_field)

load examples.pt  :./train_examples.pt, ./val_examples.pt


In [5]:
train_iter = data_handler.get_train_iter(batch_size=1)
val_iter = data_handler.get_val_iter(batch_size=1)

In [6]:
for batch in val_iter:
    input, target = batch
    break

### Embedding

这一层需要频繁的改动，所以暂时不放在py文件中

In [11]:
class EmbeddingLayer(nn.Module):
    
    def __init__(self, word_field, bert_model_path='./bert-base-uncased/', use_all=False):
        super(EmbeddingLayer, self).__init__()
        self.word_embedding_layer = nn.Embedding.from_pretrained(embeddings=word_field.vocab.vectors)
        self.word_embedding_layer.eval()
        
        model = BertModel.from_pretrained(bert_model_path)   
        self.bert_model = model
        
        self.use_all = use_all
        self.freeze()
        
    def freeze(self):
        for param in self.bert_model.parameters():
            param.requires_grad = False
        self.word_embedding_layer.weight.requires_grad = False
        
    def forward(self, word_tokens, bert_tokens):
        '''
        input:
            x: [batch_size, seg_len]
        
        return embeddings: [batch_size, seq_len, glove_dim + bert_dim]    
        '''
        word_embeddings = self.word_embedding_layer(word_tokens)
        
        # encoded_layers: [batch_size, seq_len, bert_embedding_dim] * num_of_layers
        self.bert_model.eval()
        encoded_layers, _ = self.bert_model(bert_tokens)
        
        bert_embeddings = torch.zeros_like(encoded_layers[-1])
        if self.use_all:
            for layer in encoded_layers:
                bert_embeddings += layer
            bert_embeddings /= len(encoded_layers)
        else:
            bert_embeddings += encoded_layers[-1]
        
        out = torch.cat([word_embeddings, bert_embeddings], dim=-1)
        return out
        

In [16]:
class SimpleQANet(nn.Module):
    
    def __init__(self, config, word_field):
        super(SimpleQANet, self).__init__()
        self.config = config
        self.use_cuda = config.use_cuda
        
        self.embedding_layer = EmbeddingLayer(word_field, config.bert_path, config.use_all)
        self.rnn = EncoderRNN(config.word_dim + config.bert_dim, config.hidden, 1, True, True, 0.2, False)
        
        self.qc_att = BiAttention(config.hidden*2, 0.2)
        self.linear_1 = nn.Sequential(
                nn.Linear(config.hidden*8, config.hidden),
                nn.ReLU()
        )    
        
        self.rnn_2 = EncoderRNN(config.hidden, config.hidden, 1, False, True, 0.2, False)
        
        self.self_att = SelfAttention(config.hidden*2, config.hidden*2, 0.2)       
        self.self_att_2 = SelfAttention(config.hidden*2, config.hidden*2, 0.2)        
        
        self.self_att_c = SelfAttention(config.hidden*2, config.hidden*2, 0.2)        
        
        
    def forward(self, batch):
        q_glove, _= batch.q_glove
        q_bert = batch.q_bert
        s_glove = batch.s_glove
        s_bert = batch.s_bert
        c_glove = batch.c_glove
        c_bert = batch.c_bert
        
        if self.use_cuda:
            q_glove = q_glove.cuda()
            q_bert = q_bert.cuda()
            s_glove = s_glove.cuda()
            s_bert = s_bert.cuda()
            c_glove = c_glove.cuda()
            c_bert = c_bert.cuda()
        
        # Embedding 
        q_out = self.embedding_layer(q_glove, q_bert)
        s_out = self.embedding_layer(s_glove.squeeze(), s_bert.squeeze())
        c_out = self.embedding_layer(c_glove.squeeze(), c_bert.squeeze())

        q_out = self.rnn(q_out)
        c_out = self.rnn(c_out)
        s_out = self.rnn(s_out)

        # bi-attention on supports and  question
        context_mask = (c_bert.squeeze() > 0).float()
        ques_mask = (q_bert > 0).float()
        
        support_len = s_out.size(0)
        q_out = q_out.expand(support_len, q_out.size(1), q_out.size(2))
        ques_mask = ques_mask.expand(support_len, q_out.size(1))        
        
        # s_out:[supports_len, seq_len, hidden*2], q_out: [support_len, seq_len, hidden*2]
        output = self.qc_att(s_out, q_out, ques_mask)
        output = self.linear_1(output)
        output = self.rnn_2(output)
        
        # self-attention pooling 
        # [support_len, hidden*2]
        output = self.self_att(output)
        # [1, hidden*2]
        output = self.self_att_2(output.unsqueeze(0))

        # [candidate_len, hidden*2]
        c_out = self.self_att_c(c_out)
        
        # score layer
        score = torch.mm(c_out, torch.tanh(output.transpose(0, 1)))
        return score

In [82]:
class Config:
    
    def __init__(self):
        self.hidden = 100
        self.word_dim = 300
        self.bert_dim = 768
        self.use_cuda = False
        self.bert_path = './bert-base-uncased/'
        self.use_all = True
        self.lr = 1e-4
        self.epochs = 1
        self.log_dir = './logs'
        self.model_name = 'simpleQANet'

In [83]:
config = Config()
#model = SimpleQANet(config, word_field)

In [31]:
out = model(batch)
criterion = nn.CrossEntropyLoss()
target = batch.label
out = out.transpose(0,1)
loss = criterion(out, target)

/home/zzs/Multi_Evidence_QA/model.py:150: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  alphas = self.softmax(alphas)  # (bsz, sent_len)


In [48]:
pred = out.argmax(1)

tensor([6])

In [46]:
pred.eq(target).sum().item()

0

In [28]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [72]:
def train(epoch, data_iter, model, criterion, optimizer, cuda):
    losses = AverageMeter()
    acces = AverageMeter()
    model.train()
    model.embedding_layer.eval()
    for idx, batch in enumerate(data_iter):
        score = model(batch)
        label = batch.label
        if cuda:
            label = label.cuda()
        score = score.transpose(0,1)      
        
        loss = criterion(score, label)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses.update(loss.item())
        
        pred = score.argmax(1)
        acc = pred.eq(target).sum().item()   
        acces.update(acc)
        break
        print(f'epoch:{epoch}, idx:{idx}/{len(data_iter)}, loss:{losses.avg}, acc:{acces.avg}')
    return losses.avg, acces.avg

def val(epoch, data_iter, model, criterion, cuda):
    losses = AverageMeter()
    acces = AverageMeter()
    model.eval()
    for idx, batch in enumerate(data_iter):
        with torch.no_grad():
            score = model(batch)
            
        label = batch.label
        if cuda:
            label = label.cuda()
        score = score.transpose(0,1)      
        
        loss = criterion(score, label)
        losses.update(loss.item())
        
        pred = score.argmax(1)
        acc = pred.eq(target).sum().item()   
        acces.update(acc)
        break
        print(f'epoch:{epoch}, idx:{idx}/{len(data_iter)}, loss:{losses.avg}, acc:{acces.avg}')
    return losses.avg, acces.avg

In [73]:
optimizer = torch.optim.Adam(params=filter(lambda p: p.requires_grad, model.parameters()),
                             lr=config.lr)

criterion = nn.CrossEntropyLoss()

In [ ]:
from tensorboardX import SummaryWriter

writer = SummaryWriter('')

In [74]:
for epoch in range(config.epochs):
    train_loss, train_acc = train(epoch, train_iter, model, criterion, optimizer, False)
    val_loss, val_acc = val(epoch, val_iter, model, criterion, False)
    

/home/zzs/Multi_Evidence_QA/model.py:150: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  alphas = self.softmax(alphas)  # (bsz, sent_len)


(2.9267146587371826, 0.0)

In [92]:
save_path = config.model_name + '_epoch'+str(config.epochs) + '_lr'+ str(config.lr)
save_path = os.path.join(config.log_dir, save_path)
if not os.path.exists(save_path):
    os.makedirs(save_path)